# Basic usage of the NCrystal Python API

This notebook describes the basic typical usage of the NCrystal Python API, and lays the foundation for further more detailed tutorials

TODO: Add a reference section with links to online resources (incl. the school + the other notebooks). Point out embedded help!!

## Preamble ##
Fix dependencies and tune jupyter a bit. Feel free to replace as you wish:

In [ ]:
import pathlib
import os
if 'TSL_SCHOOL_DIR' in os.environ:
    if any( (p/".git").is_dir() for p in (pathlib.Path(".").absolute().resolve()/"dummy").parents ):
        raise RuntimeError('Please copy notebook to a work directory')

In [ ]:
#Uncomment to get dependencies via pip: !pip install --quiet ipympl matplotlib

In [ ]:
%matplotlib ipympl

In [ ]:
import matplotlib
matplotlib.rcParams.update({"figure.figsize":(6.4*0.5,4.8*0.5),"figure.dpi":150,'font.size':5})

In [ ]:
%%html
<style>div.jupyter-widgets.widget-label {display: none;}</style>

Always import NCrystal of course:

In [ ]:
import NCrystal as NC
assert NC.version_num >=  3005081
NC.test() #< quick unit test that installation works!

## Using predefined materials
Jumping right into it, let us load a material defined by a *cfg-string* and plot the cross sections. Most often, a cfg-string is a data file name, and perhaps some parameters - like for setting the temperature:

In [ ]:
mat = NC.load('Al_sg225.ncmat;temp=10C')

In [ ]:
mat.plot()

The entire Python API of NCrystal is enriched with doc-strings for inline help, let us learn more about this `mat` material object:

In [ ]:
help(mat)

We can see that there are essentially `.info`, `.absorption`, and `.scatter` objects as properties as well as convenience methods `.plot()` and `.dump()`:

In [ ]:
print( mat.info )
print( mat.scatter )
print( mat.absorption )

In [ ]:
mat.dump()

The `NC.load(..)` method is merely a convenience method. If you only need a particular object you can instead call `createInfo`. `createScatter` or `createAbsorption` directly:

In [ ]:
info_Al = NC.createInfo('Al_sg225.ncmat')
info_Al.dump()

If you are interested in getting a bit more info, you can increase the verbosity (up to 2). This will affect the HKL listing and (for very complex materials) the list of atom positions:

In [ ]:
info_Al.dump(verbose=1)

For more rich plotting of cross sections, the `plot_xsects` and `plot_xsect` functions from the `NCrystal.plot` module can be used. For instance, dive into the different scattering cross section components:

In [ ]:
import NCrystal.plot as ncplot
ncplot.plot_xsect('Al_sg225.ncmat')#use show_absorption=False to hide absorption

Or perhaps you prefer macroscopic cross sections (a.k.a. inverse penetration depth). It is of course just scaled with the number density (atoms/volume) so only the Y-axis scale and text changes:

In [ ]:
ncplot.plot_xsect('Al_sg225.ncmat',xsmode='macroscopic')

If you wish to overlay different materials and compare cross sections, you have to use the `plot_xsects` (plural `s` at the end) function instead:

In [ ]:
ncplot.plot_xsects('Al_sg225.ncmat;temp=100K','Al_sg225.ncmat;temp=300K','Al_sg225.ncmat;temp=500K')

## Simple usage of Scatter or Absorption objects

If you need to extract cross section values, you can do so with the `Scatter` and `Absorption` objects (it is their *raison d'etre* after all). Note that NCrystal use electronvolt (eV) and angstrom (Å) as default units for energy and lengths respectively:

In [ ]:
scatter_Al = NC.createScatter('Al_sg225.ncmat')#NB: default temperature for most materials is 293.15K
print( '%g barn'%scatter_Al.xsect(wl=1.8) )
print( '%g eV'%NC.wl2ekin(1.8) )#NB: ekin2wl exists as well, plus many more conversion fcts in NCrystal.constants module
print( '%g barn'%scatter_Al.xsect(ekin=0.025248212841151512) )

We can use numpy arrays as well:

In [ ]:
import numpy
wavelengths = numpy.linspace(0.0,10.0,20)
print( wavelengths )
print( scatter_Al.xsect(wl=wavelengths) )

To extract single components, we must create a scatter instance with just that component, using the cfg-string parameter `comp` (+changing the temperature to show the usage of multiple parameters):

In [ ]:
scatter_Al_only_bragg = NC.createScatter('Al_sg225.ncmat;comp=bragg;temp=100K')
print( '%g barn'%scatter_Al_only_bragg.xsect(wl=4.0) )
scatter_Al_only_bragg.plot()

In addition to extracting cross sections with both `Scatter` and `Absorption` objects, one can of course also use a `Scatter` object to perform random sampling of scattering events by using the `.genscat` method. See `help(NC.Scatter)` for more information.

## Simple usage of Info objects
The `Info` object contain a range of supplementary information about the loaded material. Most notably it includes information about the material density and composition, but other information such as for instance Bragg diffraction structure factors ("HKL lists") or phonon DOS curves, can be access as well. See `help(NC.Info)` for complete details, here we will show a few examples:

In [ ]:
info = NC.createInfo('Al2O3_sg167_Corundum.ncmat')

In [ ]:
info.dump()

In [ ]:
help(info)

In [ ]:
info.composition

In [ ]:
for di in info.dyninfos:
    print(di)

In [ ]:
info.findDynInfo('Al').plot_vdos(unit='THz')#default unit is eV

In [ ]:
for ai in info.atominfos:
    print(ai)

In [ ]:
import pprint
pprint.pprint(info.findAtomInfo('Al').positions)

In [ ]:
print(info.braggthreshold)
print(info.density)
print(info.numberdensity)
print(info.sld)

In [ ]:
for iprint,(h,k,l,mult,dsp,fsq) in enumerate(info.hklList(all_indices=True)):
    print(f'FAMILY(d={dsp} fsq={fsq} mult={mult})')
    for _h,_k,_l in zip(h,k,l):
        print(f'  ----> hkl = ({_h},{_k},{_l})')
        print(f'  ----> hkl = ({-_h},{-_k},{-_l})')
    if iprint==3:
        break

## More about cfg-strings
As you have seen, a single NCrystal *cfg-string* like `"somefile.ncmat;par1=val1;par2=val2"` is all that is required to define a material in NCrystal. This is perhaps not very object-orient or pythonic, but has the advantage that the same string can be used to define a material in a multitude of contexts:

* In Python
* On the command-line
* On a webpage, in an email to a colleage, etc.
* In OpenMC
* In McStas
* In McStasScript
* In Geant4
* ...

Best of all (especially for the NCrystal developers), we can add a new cfg-string parameter without having to update all NCrystal-bindings for OpenMC/McStas/Geant4/... every time. As soon as we add the parameter in NCrystal, all users who have access to the latet NCrystal release, will automatically be able to use the new parameter.

All the parameters are described in the wiki at https://github.com/mctools/ncrystal/wiki/CfgRefDoc and the documentation is also available dynamically:

In [ ]:
print(NC.generateCfgStrDoc('txt_short'))

Or with all details:

In [ ]:
NC.generateCfgStrDoc()

### Most important cfg-parameters

* **Temperature**:
  - Examples: `temp=100`, `temp=100K`, `temp=-10C`, `temp=100F`*
  - All materials have a temperature, and it is not always 293.15! The `temp` parameter does exactly what you think it does. By default it assumes the value is in kelvin, but a unit can be added (must be one of `K`, `C`, or `F`.
* **Density**
  - Examples: `density=2.0gcm3`, `density=3.4kgm3`, `density=0.9x`.
  - Also does what you think it does. The last example scales the density by a factor of 0.9.
* **Scattering component toggling**
  - Current recognised component names are `coh_elas` (alias `bragg`), `incoh_elas`, `sans`, and `inelas`. `elas` refers to all components except `inelas`.
  - Syntax: `<compname>=0' (disable component), `comp=<compname1>,..,<compnamen>` (disable all but the listed components.
  - Examples: `
* **Modify atomic compositions**
  - Examples: `atomdb=H is D`, `atomdb=Al:is:0.9:Al:0.1:Cr`, `atomdb=Si29:28.97649466525u:4.7fm:0.001b:0.101b`, ...
* **Single crystal parameters**:
  - Single crystal models and orientations are primarily controlled by the parameters `mos`, `dir1`, `dir2`, and `dirtol`. Refer to the documentation linked aboved.
* **specialist variable needed for this school: vdoslux**
  - Controls cpu/mem/precision tradeoff in expansions of 1D phonon DOS curves to 2D scattering kernels. Higher value is more luxurious.
  - Examples: `vdoslux=5` (give me insane precision), `vdoslux=0` (only speed matters, not results).
  - Integer value 0-5, default is 3 (normal users can leave it at the default).
    (FIXME: leave out vdoslux? Do we actually use it?)
  

## The nctool (a.k.a. ncrystal_inspectfile) commandline utility
If you are in working in a terminal, the `nctool` or `ncrystal_inspectfile` command (they are the exact same thing, but `nctool` is the future-proof name) provides a convenient interface to a lot of the most common tasks an NCrystal user might need, when composing a cfg-string for their simulation:

* Load the material and do cross section (or scattering) plots or information dumps to the terminal.
  * Investigate the effect of different cfg-parameters.
* Browse available files.
* Extract content of files (including virtual files)
* Quickly unit test an installation

The `nctool` command, **like all NCrystal cmdline tools** contain embedded documentation, accessible by the `--help` flag:

In [ ]:
!nctool --help

In [ ]:
!nctool --dump "Al_sg225.ncmat;temp=200K"

In [ ]:
#UNCOMMENT IF YOU ARE OK WITH LAUNCHING NEW PLOT WINDOWS
#!nctool "Al_sg225.ncmat;temp=200K"

In [ ]:
#UNCOMMENT IF YOU ARE OK WITH LAUNCHING NEW PLOT WINDOWS
#!nctool "Be_sg194.ncmat;temp=80K" "Be_sg194.ncmat;temp=150K" "Be_sg194.ncmat;temp=300K" --energy

In [ ]:
!nctool --browse

In [ ]:
!nctool --extract Al_sg225.ncmat

## Sources of pre-defined materials
The first and foremost source, is of course materials pre-created by other scientists (be it NCrystal developers or your colleagues). Any complete NCrystal installation should include the standard data library ("stdlib") of NCMAT files. For the ~latest NCrystal release, it can be browsed online at the wiki: https://github.com/mctools/ncrystal/wiki/Data-library

So assuming you find the material (say, zirconia) you were looking for on that page, you can as usual proceed to have a look at it:

In [ ]:
import NCrystal.plot as ncplot
ncplot.plot_xsects('ZrO2_sg137_Zirconia.ncmat')#remember: room temperature by default

### Interlude: physical and virtual file data
Depending on your installation of NCrystal, the "files" in the standard data library might reside as physical files on-disk, or they might be "baked in" to the NCrystal binary library as virtual files. It should not matter to you. If you wish to get the raw content of a file, you can do so with the `createTextData` function:

In [ ]:
td = NC.createTextData('ZrO2_sg137_Zirconia.ncmat')
# or this way, to prevent accidentally picking up a file you downloaded,
# edited, and left lying around in your working directory:
# td = NC.createTextData('stdlib::ZrO2_sg137_Zirconia.ncmat')

In [ ]:
help(td)

In [ ]:
print(td.dataSourceName)
print(td.lastKnownOnDiskLocation)#will be None if file was virtual
print(td.dataType)
print('-- contents: --')
print(td.rawData)

Similarly, when you yourself need to add a new NCMAT file, you do not actually have to write a physical file into your filesystem (unless you need the file to persist after your current process is done of course). You can instead simply register the associated data as a physical file in the currently running process:

In [ ]:
a_string_with_ncmat_data="""NCMAT v7
#Don't use this material for anything
@DENSITY
  1.2345 g_per_cm3
@DYNINFO
  element  C
  fraction 1
  type     freegas
"""
NC.registerInMemoryFileData('silly_carbon.ncmat',a_string_with_ncmat_data)

In [ ]:
print(NC.createTextData('silly_carbon.ncmat').rawData)

In [ ]:
ncplot.plot_xsect('silly_carbon.ncmat;temp=200K',mode='ekin')

Of course, if all you wanted was to quickly plot the cross sections, you didn't need to register the file and invent a name for it. You could either do:

In [ ]:
mat_silly=NC.load(a_string_with_ncmat_data)
mat_silly.plot()

Or, if you really didn't need to use the material for anything else than this plot:

In [ ]:
ncplot.plot_xsect(a_string_with_ncmat_data)

### NCrystal data source infrastructure
As we are starting to see, the NCrystal infrastructure which serves up data files based on "filenames" in cfg-strings is rather flexible. The `NCrystal.datasrc` module contain many functions which can be used to fine-tune this. For instance, if you keep your own edition to a data-library in some local folder, you can add that folder to the NCrystal search path. You can either do this by setting the `NCRYSTAL_DATA_PATH` variable (*before* loading NCrystal!), which can contain multiple directories separated by semi-colons in the usual unix fashion. Or you can add a directory to this search path dynamically:

In [ ]:
extra_data_dir = pathlib.Path('./myextradatafiles/')
extra_data_dir.mkdir(exist_ok=True)
( extra_data_dir / 'my_extra_material.ncmat' ).write_text(a_string_with_ncmat_data)

In [ ]:
!ls ./myextradatafiles

In [ ]:
import NCrystal.datasrc
NCrystal.datasrc.addCustomSearchDirectory(extra_data_dir)
td = NC.createTextData('my_extra_material.ncmat')
print(td.dataSourceName)
print(td.lastKnownOnDiskLocation)#will be None if file is virtual (but it won't be in this case)
print(td.dataType)
print('-- contents: --')
print(td.rawData)

We have already seen how you can browse available data files via the command line, but you can of course also do so from Python:

In [ ]:
NC.browseFiles(dump=True)

### Gas mixtures and other on-demand NCMAT data for simple materials
As a nice side-effect from having the flexible data source infrastructure, NCrystal also contains several plugins providing "quick materials": Materials so simple, that they can be expressed in a "filename". Rather than locating a real file, the plugin handling the request will analyse the intent expressed in the "filename", and dynamically compose the appropriate NCMAT data to reflect this request. The best example of this is the `"gasmix"` plugin, handling gas mixtures:

* `"gasmix::0.72xCO2+0.28xAr/massfractions/1.5atm/250K"`
* `"gasmix::0.7xCO2+0.3xAr/0.001relhumidity"`
* `"gasmix::0.7xCO2+0.3xAr/1.5atm/250K"`
* `"gasmix::BF3/2atm/25C/B_is_0.95_B10_0.05_B11"`
* `"gasmix::CO2"`
* `"gasmix::He/1.64kgm3"`
* `"gasmix::He/10bar"`
* `"gasmix::air"`
* `"gasmix::air/-10C/0.8atm/0.30relhumidity"`

Hopefully, you can guess what each line above would give you! The only thing to be aware about though, is that unless `/massfractions` are explicitly requested as in the first example above, the mixture is going to be by-volume (or by-mole of molecules which is the same thing in this case).

You can of course always check the result by looking at the NCMAT data created in response to the request (cf. https://github.com/mctools/ncrystal/wiki/NCMAT-format). You can also load and inspect the material. First the NCMAT data:

In [ ]:
print(NC.createTextData("gasmix::0.7xCO2+0.3xAr/25C/0.001relhumidity").rawData)

Or look at the loaded `Info` object:

In [ ]:
NC.createInfo("gasmix::0.7xCO2+0.3xAr/25C/0.001relhumidity").dump()

Or plot the cross sections:

In [ ]:
ncplot.plot_xsect("gasmix::0.7xCO2+0.3xAr/25C/0.001relhumidity")

One thing you might have noticed about the dumped NCMAT data above is the section:
```
@TEMPERATURE
  298.15
```
It means that NCrystal will refuse to let the user change the temperature of this material further, which is a safeguard against someone trying to change the temperature like this (which would give a material with the wrong density!):

In [ ]:
try:
    NC.load("gasmix::0.7xCO2+0.3xAr/25C/0.001relhumidity;temp=10K")
except NC.NCBadInput as e:
    print("NCBadInput ERROR: %s"%e)

The right way to modify the temperature is inside the "filename" part (changing `25C`->`10K`):

In [ ]:
NC.load("gasmix::0.7xCO2+0.3xAr/10K/0.001relhumidity")
d=info_Al.dspacingFromHKL(5,1,1)

Two other "quick factories" exists, which can be useful in the case where a non-gaseous material needs to be included in a simulation, but where the exact material *structure* is not important (or, not known!):

* `"freegas::Ar/2.5e-5perAa3"`
* `"freegas::CF4/3.72kgm3"`
* `"freegas::CO2/1.98kgm3"`
* `"freegas::He/0.17kgm3/He_is_He3"`
* `"solid::Al2O3/4gcm3"`
* `"solid::B4C/2.52gcm3/B_is_0.95_B10_0.05_B11"`
* `"solid::CH2/1gcm3"`
* `"solid::Gd2O3/7.07gcm3"`
* `"solid::Al2O3/4gcm3/TDebye750K_Al/TDebye1000K_O"`
* `"solid::Al2O3/4gcm3/TDebye900K"`


These materials will get the density and composition which can be inferred from the strings. In the case of `freegas::` materials, all atoms will be modelled as independent free gas atoms. In a way this is no different to how materials are mostly modelled in OpenMC/Geant/MCNP/..., but it is useful to have such simple materials in NCrystal as well - not the least for when NCrystal is used standalone or in applications like McStas where there is otherwise no universal concept of a "base material".

The `solid::` materials are very similar, but they differ in that their simplistic dynamics will be appropriate for bound atoms rather than free atoms. Such a feature was easy enough to add in NCrystal, since we can generate all dynamics of an amorphous material based on a 1D phonon DOS curve, and we simply use a simplistic phonon DOS curve as input (the Debye model).

## Multiphase materials


Multiphase materials can be composed in cfg-strings from existing materials the following `"phases<...>"` cfg-string syntax, listing the desired phases and their **volume** (not mass) fraction. This is as easy as:
```
   "phases<0.1*PbS_sg225_LeadSulfide.ncmat&0.9*Epoxy_Araldite506_C18H20O3.ncmat>"
```
Cfg-parameters can be applied as always, affecting either a single phase or all phases depending on placement. For instance, in the following (somewhat silly) material the temperature of 300K applies to all phases, while the d-spacing cutoff only affects the aluminium:
```
   "phases<0.25*Al_sg225.ncmat;dcutoff=0.4&0.75*Be_sg194.ncmat>;temp=300K"
```
Note that whitespace is allowed, if you feel it provides a more readable string Also note that the `"phases<>"` syntax has been chosen specifically to support the use-case that one can always override parameters in cfg-strings by appending a simple string to them. For instance appending the string `";temp=400K"` to any cfg-string will override the temperature value - whether or not the cfg-string uses the `"phases<...>"` syntax or not.

For convenience it is also possible to define multiple phases directly inside NCMAT data. Additionally, NCrystal also has framework-level support for SANS physics, which can be thought of as diffraction due to the geometric layouts of the individual phases. We shall see an example in another notebook.

## Atom data
NCrystal obviously contains an internal database of scattering lengths, cross sections and masses, of a large number of natural elements and specific isotopes. You can access this database directly if needed:

In [ ]:
NC.atomDB('Al')

In [ ]:
NC.atomDB('He3')

You can access the data programatically, and as always you can find inline help:

In [ ]:
data_H = NC.atomDB('H')
data_D = NC.atomDB('D')
print(data_H.coherentScatLen())
print(data_D.coherentScatLen())
help(data_H)

In [ ]:
for e in NC.iterateAtomDB():
    print(e)

It is always possible to override some of these values for a particular material, or even to provide values for elements or isotopes that might be missing. This can be done either in the `@ATOMDB` section of NCMAT data (cf. https://github.com/mctools/ncrystal/wiki/NCMAT-format), or through the `atomdb` cfg-string variable (cf. https://github.com/mctools/ncrystal/wiki/CfgRefDoc). We will return to this subject when discussing the `NCMATComposer` in another Notebook.

## References and where to find more information

The available documentation for NCrystal is a work in progress, and while we still didn't consolidate this in one glorious location, there is plenty of information to be found:

* The official wiki: https://github.com/mctools/ncrystal/wiki/
  * This includes the [Data-library](https://github.com/mctools/ncrystal/wiki/Data-library) and [CfgRefDoc](https://github.com/mctools/ncrystal/wiki/CfgRecDoc) pages which most users might need.
  * For advanced users, the offical [NCMAT format specification](https://github.com/mctools/ncrystal/wiki/NCMAT-format) is there as well, along with instructions for usage or development of optional [plugins](https://github.com/mctools/ncrystal/wiki/Plugins).
  * Be sure to check out the various release Announcement pages you find in the sidebar, since a lot of information was described there but still didn't make it anywhere else.
* The Python API contains doc-strings everywhere, accessible though the `help` function. Please use it!
* All command-line tools support a `--help` flag, which will result in usage instructions being printed out.
* For C++ developers: Lots and lots of thorough comments are left everywhere in the code.
* The [README](https://github.com/mctools/ncrystal/blob/master/README) and [INSTALL](https://github.com/mctools/ncrystal/blob/master/INSTALL) files from the NCrystal release itself.
* Publications (also, **please remember to cite if you use NCrystal and want to support us!**):
  * X.-X. Cai and T. Kittelmann, NCrystal: A library for thermal neutron transport, Computer Physics Communications 246 (2020) 106851, https://doi.org/10.1016/j.cpc.2019.07.015
  * T. Kittelmann and X.-X. Cai, Elastic neutron scattering models for NCrystal, Computer Physics Communications 267 (2021) 108082, https://doi.org/10.1016/j.cpc.2021.108082
  * X.-X. Cai, T. Kittelmann, et. al., "Rejection-based sampling of inelastic neutron scattering", Journal of Computational Physics 380 (2019) 400-407, https://doi.org/10.1016/j.jcp.2018.11.043

## How to get help

There are many options for how to get help with NCrystal. We welcome all contact, but given that we are normally stretched for manpower, some means of contact are certainly easier for us to deal with than others!

* **Using the discussion forum for questions or discussions**
  * Our new discussions forum at https://github.com/mctools/ncrystal/discussions is where we hope that users of all levels will ask their questions!
  * Do not be afraid to ask "stupid questions". Such questions are rarely actually stupid, and but often expose the need for better documentation, or real bugs in our code. Or perhaps all there is needed is another NCrystal user to share the solution they had in a similar situation.
  * Should a question end up exposing a bug (or feature request) for NCrystal, we can also easily convert them to proper GitHub "issues", and include other relevant experts in the discussions.
  * Feel free to use the forum for other NCrystal-related subjects as well.
  * **Please sign up to "watch" the discussions if you are an NCrystal user that want to contribute by helping other users**
* **Reporting bugs or feature requests**
  * Please (please!) do so using the tracker at https://github.com/mctools/ncrystal/issues
  * Don't worry if you don't get the meta-data of the request correct, we can fix it up later.
  * Do try to include as many details as possible about your issue, so we don't have to drag them out of you.
* **Sending emails to NCrystal developers**
  * Of course you can do so, but please (please!) consider first if you could not use one of the other manner of contacts above.
  * Emails can be very distracting, since in practice they must be dealt with either "immediately" or "never".
  * We will never be upset at getting emails, but you might get a short reply asking you to post your problem in one of the GitHub channels instead.
